<a href="https://colab.research.google.com/github/ikyath/RNN_Introduction_Fundamentals/blob/master/RNN_Theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction


Time Series

A time series is a series of data points indexed (or listed or graphed) in time order. Most commonly, a time series is a sequence taken at successive equally spaced points in time. Thus it is a sequence of discrete-time data











Example - Weather

Weather is a dynamical System

  See : Chaos Theory,Butterfly effect

![alt text](https://i.ytimg.com/vi/D6jln7pqn70/maxresdefault.jpg)

This is that even if we have the exact deterministic equations to describe a weather system our forecast

will still become more and more incorrect.

The further into the future we try to predict that's pretty counterintuitive because you would think

if you have the exact equation for something then you should be able to calculate all the future values

precisely but due to the butterfly effect this is not actually true.

As the saying goes a butterfly flapping its wings in Tokyo can cause a tornado in America.

Small and decisions like numerical round off error and your computer will ultimately lead it to your

weather forecast being completely wrong eventually.

This is actually really relevant to us because when you think of time series and our own ends you automatically

![alt text](https://media.graytvinc.com/images/810*455/lnk+wx1.jpg)




















